In [1]:
from tdc import Oracle

In [2]:
import torch
from torchdrug import datasets
dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True, atom_feature="symbol")

Loading /Users/ca/molecule-datasets/250k_rndm_zinc_drugs_clean_3.csv:  50%|█████     | 249456/498911 [00:03<00:03, 71262.15it/s]
Constructing molecules from SMILES: 100%|██████████| 249455/249455 [06:23<00:00, 650.55it/s]


In [50]:
from numpy import max

In [48]:
ASA = Oracle(name = 'SA')

In [67]:
ls = [1,2,3,4,1,1]
print(sorted(ls)[:3])

[1, 1, 1]
